In [17]:
import qiskit
import qiskit_superstaq as qss

from applications_superstaq import ResourceEstimate
from qiskit.circuit.random import random_circuit

# Getting Resource Estimate Costs

In [18]:
# Connect to SuperstaQ
superstaq = qss.superstaq_provider.SuperstaQProvider(
    "YOUR API KEY",
    remote_host=qss.API_URL,
)

# Resource Estimate for Simple Circuit

In [19]:
# Create simple circuit and submit to Neutral Atom Device
circuit = qiskit.QuantumCircuit(2)
circuit.cnot(0, 1)
circuit.h(1)

circuit.draw()

q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ H ├
     └───┘└───┘

In [20]:
resource_estimate = superstaq.resource_estimate(circuit, "neutral_atom_qpu")
print(resource_estimate)

ResourceEstimate(num_single_qubit_gates=1, num_two_qubit_gates=1, depth=2)


# Resource Estimate for More Complex Random Circuit

In [21]:
# Create and submit random circuit to Neutral Atom Device
circuit_random = random_circuit(4, 4, measure=True)
circuit_random.draw()

┌───┐                                      ┌─┐      
q_0: ───────────────X─┤ X ├────────────────────X──■──────────────┤M├──────
     ┌────────────┐ │ └─┬─┘┌────────────┐┌───┐ │  │              └╥┘┌─┐   
q_1: ┤ Ry(5.0101) ├─┼───┼──┤ Ry(5.4424) ├┤ Y ├─┼──┼──■────────────╫─┤M├───
     └───┬───┬────┘ │   │  └────────────┘└─┬─┘ │  │  │ZZ(4.8717)  ║ └╥┘┌─┐
q_2: ────┤ I ├──────┼───■──────────────────┼───X──┼──■────────────╫──╫─┤M├
         └───┘      │ ┌───┐                │      │      ┌─┐      ║  ║ └╥┘
q_3: ───────────────X─┤ S ├────────────────■──────■──────┤M├──────╫──╫──╫─
                      └───┘                              └╥┘      ║  ║  ║ 
c: 4/═════════════════════════════════════════════════════╩═══════╩══╩══╩═
                                                          3       0  1  2

In [22]:
resource_estimate = superstaq.resource_estimate(circuit_random, "neutral_atom_qpu")
print(resource_estimate)

ResourceEstimate(num_single_qubit_gates=8, num_two_qubit_gates=6, depth=5)


# Example on Circuit Compiled on IBMQ Device

In [23]:
circuit2 = qiskit.QuantumCircuit(2)
circuit2.swap(0, 1)
circuit2.draw()

q_0: ─X─
      │ 
q_1: ─X─

In [26]:
out = superstaq.ibmq_compile(circuit2, target="ibmq_jakarta_qpu")
print(out.circuit)

     ┌─────────┐┌───────────┐┌─────────┐ ┌─────────┐┌───────┐┌───────────┐»
q_0: ┤ Rz(π/2) ├┤0          ├┤ Rz(π/2) ├─┤ Rx(π/2) ├┤ Rz(π) ├┤0          ├»
     └─────────┘│  Acecr_mp │├─────────┴┐├─────────┤└───────┘│  Acecr_pm │»
q_1: ───────────┤1          ├┤ Rz(3π/2) ├┤ Rx(π/2) ├─────────┤1          ├»
                └───────────┘└──────────┘└─────────┘         └───────────┘»
«     ┌──────────┐┌─────────┐┌───────┐┌───────────┐
«q_0: ┤ Rz(3π/2) ├┤ Rx(π/2) ├┤ Rz(π) ├┤0          ├
«     ├──────────┤└─────────┘└───────┘│  Acecr_pm │
«q_1: ┤ Rz(3π/2) ├────────────────────┤1          ├
«     └──────────┘                    └───────────┘


In [29]:
resource_estimate = superstaq.resource_estimate(circuit2, "ibmq_jakarta_qpu")
print(resource_estimate)

ResourceEstimate(num_single_qubit_gates=10, num_two_qubit_gates=3, depth=10)


# Multiple Circuit Example

In [34]:
# Create List of random circuits (3)
circuits = [
    random_circuit(2, 2, measure=True),
    random_circuit(3, 3, measure=True),
    random_circuit(4, 4, measure=True),
]
for c in circuits:
    print(c)

                   ┌─────────────┐┌─┐   
q_0: ─■────────────┤ Rz(0.18388) ├┤M├───
      │U1(0.91908) └──────┬──────┘└╥┘┌─┐
q_1: ─■───────────────────■────────╫─┤M├
                                   ║ └╥┘
c: 2/══════════════════════════════╩══╩═
                                   0  1 
     ┌───┐                                        ┌─┐   
q_0: ┤ T ├─────────────■────────────────■─────────┤M├───
     └───┘┌────────────┴─────────────┐  │  ┌─────┐└╥┘┌─┐
q_1: ──■──┤ U3(2.6067,5.5509,5.4419) ├──┼──┤ Tdg ├─╫─┤M├
       │  └──────────┬───┬───────────┘┌─┴─┐└─┬─┬─┘ ║ └╥┘
q_2: ──■─────────────┤ Y ├────────────┤ X ├──┤M├───╫──╫─
                     └───┘            └───┘  └╥┘   ║  ║ 
c: 3/═════════════════════════════════════════╩════╩══╩═
                                              2    0  1 
                                                      ┌───┐     ┌─┐         
q_0: ─X────────────────────────────■──────────────────┤ X ├──■──┤M├─────────
      │        ┌─────┐       ┌───┐ │          

In [36]:
resource_estimates = superstaq.resource_estimate(circuits, "neutral_atom_qpu")
print(resource_estimates)

[ResourceEstimate(num_single_qubit_gates=2, num_two_qubit_gates=2, depth=3), ResourceEstimate(num_single_qubit_gates=7, num_two_qubit_gates=3, depth=5), ResourceEstimate(num_single_qubit_gates=11, num_two_qubit_gates=3, depth=5)]
